In [1]:
from datasets import load_dataset, DatasetDict, load_from_disk, load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

In [2]:
import datasets
datasets.__version__

'1.18.4'

In [3]:
# local = load_from_disk("../data/imdb")
# local

In [4]:
imdb = load_dataset("../output/huggingface/imdb")
imdb

Using custom data configuration redacted--imdb-5aaa0cb7c83e1b39
Reusing dataset parquet (/Users/user/.cache/huggingface/datasets/parquet/redacted--imdb-5aaa0cb7c83e1b39/0.0.0/0b6d5799bb726b24ad7fc7be720c170d8e497f575d02d47537de9a5bac074901)


  0%|          | 0/4 [00:00<?, ?it/s]

DatasetDict({
    dev: Dataset({
        features: ['text', 'label'],
        num_rows: 1000
    })
    attack_eval_truncated: Dataset({
        features: ['text', 'label'],
        num_rows: 500
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 24000
    })
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
})

In [5]:
def truncate(example, max_words=110):
    text = example['text']
    label = example['label']
    
    text = text.split()
    if len(text) > max_words:
        text = text[:max_words]
    
    text = " ".join(text)
    return {'text': text}



In [7]:
_, attack_eval = imdb['test'].train_test_split(test_size=500,seed=31).values()
attack_eval

Dataset({
    features: ['text', 'label'],
    num_rows: 500
})

In [8]:
attack_eval = attack_eval.map(truncate)

0ex [00:00, ?ex/s]

In [9]:
attack_eval

Dataset({
    features: ['text', 'label'],
    num_rows: 500
})

In [22]:
len(attack_eval[378]['text'].split())

110

In [23]:
attack_eval[378]

{'text': "RAINBOW BRITE AND THE STAR STEALER, in my opinion, is an excellent masterpiece. I felt all warm and tingly when Rainbow Brite (voice of Bettina) and Krys (voice of David Mendenhall) set out to get revenge on the princess (voice of Rhonad Aldrich). If you ask me, the princess was absolutely bitchy and diabolical. To me, she deserved to have Rainbow Brite and Krys seek revenge on him. However, I liked her castle as well as the rest of the setting. In addition, I thought that Lurky (voiec of Pat Fraley) and Murky (voiec of Peter Cullen) were diabolical. Before I wrap this up, I'd like to say that everyone",
 'label': 1}

In [20]:
imdb= DatasetDict({"train":imdb['train'],
             "dev":imdb['dev'],
             "test":imdb['test'],
             "attack_eval_truncated":attack_eval                
            })

In [21]:
imdb

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    dev: Dataset({
        features: ['text', 'label'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 24000
    })
    attack_eval_truncated: Dataset({
        features: ['text', 'label'],
        num_rows: 500
    })
})

In [24]:
from huggingface_hub import list_datasets

all_datasets = list_datasets()
print(f"Number of datasets on Hub: {len(all_datasets)}")
print(all_datasets[0])

Number of datasets on Hub: 3820
Dataset Name: acronym_identification, Tags: ['annotations_creators:expert-generated', 'language_creators:found', 'languages:en', 'licenses:mit', 'multilinguality:monolingual', 'size_categories:10K<n<100K', 'source_datasets:original', 'task_categories:structure-prediction', 'task_ids:structure-prediction-other-acronym-identification', 'pretty_name:Acronym Identification Dataset']


In [33]:
from huggingface_hub import notebook_login

In [34]:
from huggingface_hub import create_repo

repo_url = create_repo(name="imdb", repo_type="dataset")
repo_url

HTTPError: 409 Client Error: Conflict for url: https://huggingface.co/api/repos/create - You already created this dataset repo

In [35]:
imdb.push_to_hub("../output/huggingface/imdb")

Pushing split train to the Hub.
The repository already exists: the `private` keyword argument will be ignored.


Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Pushing split dev to the Hub.
The repository already exists: the `private` keyword argument will be ignored.


Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Pushing split test to the Hub.
The repository already exists: the `private` keyword argument will be ignored.


Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Pushing split attack_eval_truncated to the Hub.
The repository already exists: the `private` keyword argument will be ignored.


Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

In [36]:
imdb

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    dev: Dataset({
        features: ['text', 'label'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 24000
    })
    attack_eval_truncated: Dataset({
        features: ['text', 'label'],
        num_rows: 500
    })
})

In [32]:
imdb.save_to_disk("../data/imdb2")